In [ ]:
# Date: 30.11.24
# Purpose: Question LLM, have it make API call and return result
# Theme: Tools/function use
# Status: Yet to start

In [3]:
# Not certain this is needed.
pip install google-generativeai

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Tim_S\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
# Validate i can call the llm
import os
import google.generativeai as genai

genai.configure(api_key= )

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
)

chat_session = model.start_chat(
  history=[
  ]
)

response = chat_session.send_message("Where is Paris")
print(response.text)

Paris is located in **France**, in the northern part of the country, on the River Seine.



In [14]:
# Now need an API to call:
# This looks good: https://open-meteo.com/en/docs#latitude=-33.8678&longitude=151.2073

#!pip install openmeteo-requests
#!pip install requests-cache retry-requests
# # numpy pandas

Defaulting to user installation because normal site-packages is not writeable
  Using cached requests_cache-1.2.1-py3-none-any.whl.metadata (9.9 kB)
  Using cached cattrs-24.1.2-py3-none-any.whl.metadata (8.4 kB)
Using cached requests_cache-1.2.1-py3-none-any.whl (61 kB)
Using cached cattrs-24.1.2-py3-none-any.whl (66 kB)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\Tim_S\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Version 1 (POC)

In [95]:
# Validate i can call the api
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": -33.8678,
	"longitude": 151.2073,
	"hourly": "temperature_2m"
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
display(hourly_dataframe.head(1))
df = hourly_dataframe.head(1)

result = ' | '.join(df['date'].dt.strftime('%Y-%m-%d %H:%M:%S %Z') + " and " + df['temperature_2m'].astype(str))
print(result)

# test for later
test = params

Coordinates -33.75°N 151.125°E
Elevation 51.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m
0,2024-11-30 00:00:00+00:00,22.309


2024-11-30 00:00:00 UTC and 22.309


In [ ]:
# Need to validate i get the llm to output json, that can be used to call the api
import os
import google.generativeai as genai

genai.configure(api_key= )

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
)

chat_session = model.start_chat(
  history=[
  ]
)

# Below worked.
# response = chat_session.send_message("""can you please provide me the latitude and longitude or brisbane? \
#                                      Please make sure outout is exactly as follows \
#                                      "{
# 	"latitude": ,
# 	"longitude": ,
# 	"hourly": "temperature_2m"
# }"
#                                     """)

# Copy of above, but trialing new location
response = chat_session.send_message("""can you please provide me the latitude and longitude or melbourne? \
                                     Please make sure outout is exactly as follows \
                                     "{
	"latitude": ,
	"longitude": ,
	"hourly": "temperature_2m"
}"
                                     """)

print(response.text)
params1 = (response.text)

# Process outout to make sure it is correct structure (braketed content) & format (json)
# -- 1.0 Structure (xtract the part inside the curly brackets)
extracted = params1.split('{', 1)[1].rsplit('}', 1)[0]
# Add the brackets back to make it valid JSON
json_data = '{' + extracted + '}'
#print(json_data)

# -- 2.0 Format (json)
import json
try:
    # Attempt to parse the JSON string
    params = json.loads(json_data)
    print("Valid JSON:", params)
except json.JSONDecodeError as e:
    # Handle the error if the string is not valid JSON
    print("Invalid JSON:", e)

```json
{
	"latitude": -37.8136,
	"longitude": 144.9631,
	"hourly": "temperature_2m"
}
```

Valid JSON: {'latitude': -37.8136, 'longitude': 144.9631, 'hourly': 'temperature_2m'}


In [125]:
# Was WIP, seemed to work, moved to above
# # import json
# # params = json.loads(response.text)
# #print(params1)

# # Extract the part inside the curly brackets
# extracted = params1.split('{', 1)[1].rsplit('}', 1)[0]
# # Add the brackets back to make it valid JSON
# json_data = '{' + extracted + '}'
# print(json_data)

# import json
# try:
#     # Attempt to parse the JSON string
#     params = json.loads(json_data)
#     print("Valid JSON:", params)
# except json.JSONDecodeError as e:
#     # Handle the error if the string is not valid JSON
#     print("Invalid JSON:", e)

In [139]:
# Now test if the above can call the API. IF yes, then just need to combine above and this...

# Validate i can call the api
import openmeteo_requests
import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"


#params = test
params = params
# ----- The below is replaced by the llm call ---
# -----------------------------------------------
# params = {
# 	"latitude": -33.8678,
# 	"longitude": 151.2073,
# 	"hourly": "temperature_2m"
# }

# -----------------------------------------------

responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["temperature_2m"] = hourly_temperature_2m

hourly_dataframe = pd.DataFrame(data = hourly_data)
display(hourly_dataframe.head(1))
df = hourly_dataframe.head(1)

result = ' | '.join(df['date'].dt.strftime('%Y-%m-%d %H:%M:%S %Z') + " and " + df['temperature_2m'].astype(str))
print(result)

Coordinates -37.75°N 144.875°E
Elevation 19.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m
0,2024-12-01 00:00:00+00:00,22.667


2024-12-01 00:00:00 UTC and 22.667


In [137]:
# QA above using the source link
# Source: https://open-meteo.com/en/docs#latitude=-37.814&longitude=144.9633


import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": -37.814,
	"longitude": 144.9633,
	"hourly": ["is_day", "sunshine_duration", "cape"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_is_day = hourly.Variables(0).ValuesAsNumpy()
hourly_sunshine_duration = hourly.Variables(1).ValuesAsNumpy()
hourly_cape = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["is_day"] = hourly_is_day
hourly_data["sunshine_duration"] = hourly_sunshine_duration
hourly_data["cape"] = hourly_cape

hourly_dataframe = pd.DataFrame(data = hourly_data)

display(hourly_dataframe.head(1))


Coordinates -37.75°N 144.875°E
Elevation 19.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,is_day,sunshine_duration,cape
0,2024-12-01 00:00:00+00:00,1.0,3600.0,0.0


## Version 2 (trying to pull different data from API)

In [ ]:
# Need to validate i get the llm to output json, that can be used to call the api
import os
import google.generativeai as genai

genai.configure(api_key= )

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
)

chat_session = model.start_chat(
  history=[
  ]
)

# Below worked.
# response = chat_session.send_message("""can you please provide me the latitude and longitude or brisbane? \
#                                      Please make sure outout is exactly as follows \
#                                      "{
# 	"latitude": ,
# 	"longitude": ,
# 	"hourly": "temperature_2m"
# }"
#                                     """)

# Copy of above, but trialing new location
response = chat_session.send_message("""can you please provide me the latitude and longitude or new york city? \
                                     Please make sure outout is exactly as follows \
                                     "{
	"latitude": ,
	"longitude": ,
	"hourly": ["is_day", "sunshine_duration", "cape"]
}"
                                     """)



print(response.text)
params1 = (response.text)

# Process outout to make sure it is correct structure (braketed content) & format (json)
# -- 1.0 Structure (xtract the part inside the curly brackets)
extracted = params1.split('{', 1)[1].rsplit('}', 1)[0]
# Add the brackets back to make it valid JSON
json_data = '{' + extracted + '}'
#print(json_data)

# -- 2.0 Format (json)
import json
try:
    # Attempt to parse the JSON string
    params = json.loads(json_data)
    print("Valid JSON:", params)
except json.JSONDecodeError as e:
    # Handle the error if the string is not valid JSON
    print("Invalid JSON:", e)

```json
{
	"latitude": 40.7128,
	"longitude": -74.0060,
	"hourly": ["is_day", "sunshine_duration", "cape"]
}
```

Valid JSON: {'latitude': 40.7128, 'longitude': -74.006, 'hourly': ['is_day', 'sunshine_duration', 'cape']}


In [145]:
# Now test if the above can call the API. IF yes, then just need to combine above and this...

# Validate i can call the api
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"


params = params
# params = {
# 	"latitude": -37.814,
# 	"longitude": 144.9633,
# 	"hourly": ["is_day", "sunshine_duration", "cape"]
# }
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_is_day = hourly.Variables(0).ValuesAsNumpy()
hourly_sunshine_duration = hourly.Variables(1).ValuesAsNumpy()
hourly_cape = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["is_day"] = hourly_is_day
hourly_data["sunshine_duration"] = hourly_sunshine_duration
hourly_data["cape"] = hourly_cape

hourly_dataframe = pd.DataFrame(data = hourly_data)
display(hourly_dataframe.head(1))



Coordinates 40.71033477783203°N -73.99308776855469°E
Elevation 32.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,is_day,sunshine_duration,cape
0,2024-12-01 00:00:00+00:00,0.0,0.0,0.0


In [143]:
# QA above:
# Now test if the above can call the API. IF yes, then just need to combine above and this...

# Validate i can call the api

import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://api.open-meteo.com/v1/forecast"
params = {
	"latitude": 40.7143,
	"longitude": -74.006,
	"hourly": ["is_day", "sunshine_duration", "cape"]
}
responses = openmeteo.weather_api(url, params=params)

# Process first location. Add a for-loop for multiple locations or weather models
response = responses[0]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_is_day = hourly.Variables(0).ValuesAsNumpy()
hourly_sunshine_duration = hourly.Variables(1).ValuesAsNumpy()
hourly_cape = hourly.Variables(2).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}
hourly_data["is_day"] = hourly_is_day
hourly_data["sunshine_duration"] = hourly_sunshine_duration
hourly_data["cape"] = hourly_cape

hourly_dataframe = pd.DataFrame(data = hourly_data)
display(hourly_dataframe.head(1))

Coordinates 40.71033477783203°N -73.99308776855469°E
Elevation 51.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,is_day,sunshine_duration,cape
0,2024-12-01 00:00:00+00:00,0.0,0.0,0.0


## Version 3 (trying different API)

In [ ]:
#pip install yfinance
# Code and idea from ChatGTP, i.e. 
    # Yahoo Finance API (via yfinance library)
    # Details: Yahoo Finance is a popular source for historical stock prices and financial data.
    # Access: Free and doesn't require an API key.
    # How to Use: Use the Python yfinance library to fetch data easily.
    # Example:
        # import yfinance as yf
        # # Download historical data for a stock
        # data = yf.download("AAPL", start="2022-01-01", end="2022-12-31")
        # print(data)
        # Pros: Comprehensive, reliable, and easy to use.
        # Cons: Rate-limited; excessive requests may lead to blocking.

In [150]:
# Validate can make API all (albeit somewhat abstracted)
import yfinance as yf
# Download historical data for a stock
data = yf.download("AAPL", start="2022-01-01", end="2022-12-31")
display(data.head(3))

[*********************100%***********************]  1 of 1 completed

Price,Adj Close,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,,
2022-01-03,179.076599,182.009995,182.880005,177.710007,177.830002,104487900
2022-01-04,176.803802,179.699997,182.940002,179.119995,182.630005,99310400
2022-01-05,172.100830,174.919998,180.169998,174.639999,179.610001,94537600


In [ ]:
# Now get the LLM to source the data
# Validate i can call the llm
import os
import google.generativeai as genai

genai.configure(api_key= )

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 40,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}

model = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
)

chat_session = model.start_chat(
  history=[
  ]
)
question = "Can you please provide the stock price for AAPL?, for the previous week?"
# instruction = """Please extract the stock code (4 character code) and date range from the question. \
#                If no date range provide list start date and end date as yesterdays date \
#                If date range provided use dates provided as start and end date.\
#                Please make sure outout is exactly as follows \
#                '"{stock code}", start="{start date}", end="{end date}"'"""

instruction = """Please extract the stock code (4 character code) and date range from the question. \
               If no date range provide list start date and end date as yesterdays date \
               If date range provided use dates provided as start and end date.\
               Please make sure outout is exactly as follows \
               ticker = 
               start_date = 
               end_date = 
'"""

#"AAPL", start="2022-01-01", end="2022-12-31"
print("targ output:", '''"AAPL", start="2022-01-01", end="2022-12-31"''')

response = chat_session.send_message(instruction + question)
#print("llm output:", response.text)
llm = (response.text)
print("llm output :", llm)



# # Output
# # Download historical data for a stock
# #data = yf.download("AAPL", start="2022-01-01", end="2022-12-31")
# data = yf.download(llm)
# display(data.head(3))

targ output: "AAPL", start="2022-01-01", end="2022-12-31"
llm output : ticker = AAPL
start_date = 2023-10-26
end_date = 2023-11-01



: 

In [183]:
# Output
# Download historical data for a stock
#data = yf.download("AAPL", start="2022-01-01", end="2022-12-31")
#data = yf.download("AAPL", start="2023-10-26", end="2023-10-31")
data = yf.download(s)
display(data.head(3))

[*********************100%***********************]  2 of 3 completed

3 Failed downloads:
['END="2023-10-30"', '"AAPL"', 'START="2023-10-26"']: YFTzMissingError('$%ticker%: possibly delisted; no timezone found')


Empty DataFrame
Columns: [(Adj Close, "AAPL"), (Adj Close, END="2023-10-30"), (Adj Close, START="2023-10-26"), (Close, "AAPL"), (Close, END="2023-10-30"), (Close, START="2023-10-26"), (High, "AAPL"), (High, END="2023-10-30"), (High, START="2023-10-26"), (Low, "AAPL"), (Low, END="2023-10-30"), (Low, START="2023-10-26"), (Open, "AAPL"), (Open, END="2023-10-30"), (Open, START="2023-10-26"), (Volume, "AAPL"), (Volume, END="2023-10-30"), (Volume, START="2023-10-26")]
Index: []

In [162]:
print('''"AAPL", start="2022-01-01", end="2022-12-31"''')
print(llm)

"AAPL", start="2022-01-01", end="2022-12-31"
("AAPL", start="2023-10-26", end="2023-10-30")



In [184]:
import yfinance as yf

# Define variables for each argument
ticker = "AAPL"
start_date = "2023-10-26"
end_date = "2023-10-31"

# Pass the variables directly to yf.download()
data = yf.download(ticker, start=start_date, end=end_date)

print(data)

[*********************100%***********************]  1 of 1 completed

Price        Adj Close       Close        High         Low        Open  \
Ticker            AAPL        AAPL        AAPL        AAPL        AAPL   
Date                                                                     
2023-10-26  165.857880  166.889999  171.380005  165.669998  170.369995   
2023-10-27  167.179657  168.220001  168.960007  166.830002  166.910004   
2023-10-30  169.236847  170.289993  171.169998  168.869995  169.020004   

Price         Volume  
Ticker          AAPL  
Date                  
2023-10-26  70625300  
2023-10-27  58499100  
2023-10-30  51131000  
